In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format='retina'
from sklearn import preprocessing
from sklearn.cluster import KMeans

In [2]:
print(f'pandas   version = {pd.__version__}')
print(f'numpy    version = {np.__version__}')
print(f'seaborn  version = {sns.__version__}')

pandas   version = 1.2.4
numpy    version = 1.20.3
seaborn  version = 0.11.1


In [3]:
#file ='E:\90 Nida\7105 CRM\06 Homework06 - Customer Segmentation/SupermarketData.csv'
df=pd.read_csv(r'E:\90 Nida\7105 CRM\06 Homework06 - Customer Segmentation/SupermarketData.csv')
#df.head(10)

In [4]:
# Check missing value
df.isnull().sum()

SHOP_WEEK                        0
SHOP_DATE                        0
SHOP_WEEKDAY                     0
SHOP_HOUR                        0
QUANTITY                         0
SPEND                            0
PROD_CODE                        0
PROD_CODE_10                     0
PROD_CODE_20                     0
PROD_CODE_30                     0
PROD_CODE_40                     0
CUST_CODE                   153682
CUST_PRICE_SENSITIVITY      153682
CUST_LIFESTAGE              259315
BASKET_ID                        0
BASKET_SIZE                      0
BASKET_PRICE_SENSITIVITY         0
BASKET_TYPE                      0
BASKET_DOMINANT_MISSION          0
STORE_CODE                       0
STORE_FORMAT                     0
STORE_REGION                     0
dtype: int64

In [5]:
#Seperate one-time customer group
df['CUST_CODE'] = df['CUST_CODE'].fillna("X")
df1 = df[df['CUST_CODE'] == "X"]
df1.head(1000000)

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
4,200737,20071110,7,14,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,NaN,NaN,994108300002212,L,MM,Full Shop,Fresh,STORE00001,LS,E02
10,200813,20080525,1,18,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,NaN,NaN,994111100127694,L,MM,Top Up,Fresh,STORE00001,LS,E02
14,200747,20080117,5,15,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,NaN,NaN,994109300126156,L,LA,Top Up,Fresh,STORE00001,LS,E02
16,200731,20070928,6,21,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,NaN,NaN,994107700039395,L,LA,Top Up,Fresh,STORE00001,LS,E02
18,200747,20080119,7,20,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,NaN,NaN,994109300051381,L,LA,Full Shop,Mixed,STORE00001,LS,E02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956557,200618,20060627,3,21,3,3.96,PRD0904997,CL00074,DEP00021,G00007,...,NaN,NaN,994101200022574,M,LA,Small Shop,Fresh,STORE00002,LS,W01
956559,200617,20060621,4,17,3,3.96,PRD0904997,CL00074,DEP00021,G00007,...,NaN,NaN,994101100001073,L,LA,Top Up,Fresh,STORE00002,LS,W01
956569,200617,20060622,5,12,3,3.96,PRD0904997,CL00074,DEP00021,G00007,...,NaN,NaN,994101100088778,M,MM,Top Up,Fresh,STORE00002,LS,W01
956570,200633,20061013,6,20,3,3.96,PRD0904997,CL00074,DEP00021,G00007,...,NaN,NaN,994102700099738,L,LA,Top Up,Fresh,STORE00002,LS,W01


In [6]:
#Seperate membership customer group
df2 = df[df['CUST_CODE'] != "X"]
df2.head(1000000)

,SHOP_WEEK,SHOP_DATE,SHOP_WEEKDAY,SHOP_HOUR,QUANTITY,SPEND,PROD_CODE,PROD_CODE_10,PROD_CODE_20,PROD_CODE_30,...,CUST_PRICE_SENSITIVITY,CUST_LIFESTAGE,BASKET_ID,BASKET_SIZE,BASKET_PRICE_SENSITIVITY,BASKET_TYPE,BASKET_DOMINANT_MISSION,STORE_CODE,STORE_FORMAT,STORE_REGION
0,200732,20071005,6,17,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,UM,YF,994107800547472,L,MM,Top Up,Grocery,STORE00001,LS,E02
1,200733,20071010,4,20,3,6.75,PRD0900001,CL00072,DEP00021,G00007,...,MM,OF,994107900512001,L,MM,Full Shop,Fresh,STORE00001,LS,E02
2,200741,20071209,1,11,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,MM,YF,994108700468327,L,MM,Full Shop,Grocery,STORE00001,LS,E02
3,200731,20070929,7,17,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,LA,OF,994107700237811,L,LA,Full Shop,Mixed,STORE00001,LS,E02
5,200746,20080108,3,15,1,2.25,PRD0900001,CL00072,DEP00021,G00007,...,MM,NaN,994109200504187,L,MM,Top Up,Fresh,STORE00001,LS,E02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956566,200633,20061009,2,12,3,3.96,PRD0904997,CL00074,DEP00021,G00007,...,LA,OT,994102700591584,L,LA,Full Shop,Mixed,STORE00002,LS,W01
956567,200634,20061022,1,16,3,3.96,PRD0904997,CL00074,DEP00021,G00007,...,LA,OA,994102800730797,L,LA,Top Up,Grocery,STORE00002,LS,W01
956568,200618,20060701,7,12,3,3.96,PRD0904997,CL00074,DEP00021,G00007,...,LA,OA,994101200777910,L,LA,Top Up,Fresh,STORE00002,LS,W01
956571,200617,20060622,5,18,3,3.96,PRD0904997,CL00074,DEP00021,G00007,...,LA,YA,994101100506174,L,LA,Top Up,Fresh,STORE00002,LS,W01


In [7]:
#Feature Engineering by check correlation
#df1.describe().round(2)
#df1.hist(bins=20, figsize=(12,6))
#plt.show()
#sns.pairplot(df1)
#plt.show()
#plt.boxplot(df1.QUANTITY)
#plt.show()

In [8]:
#plt.figure(figsize=(6,4.5))
#cor = df.corr()
#g = sns.heatmap(cor, annot=True, cmap='YIGnBu', linewidths=0.8, linecolor='b', fnt='.2f')
#g.set_yticklabels(g.get_yticklabels(), rotation=0, fontsize=12)
#plt.show()

In [9]:
"""
Scikit-learn : KMeans Clustering
1.Scale data
2.Optimal number of clusters
    A.Silhouette_analysis
        a.visualize silhouette
    B.Elbow method
3.Compute and name clusters
"""

'\nScikit-learn : KMeans Clustering\n1.Scale data\n2.Optimal number of clusters\n    A.Silhouette_analysis\n        a.visualize silhouette\n    B.Elbow method\n3.Compute and name clusters\n'

In [11]:
df2.columns

Index(['SHOP_WEEK', 'SHOP_DATE', 'SHOP_WEEKDAY', 'SHOP_HOUR', 'QUANTITY',
       'SPEND', 'PROD_CODE', 'PROD_CODE_10', 'PROD_CODE_20', 'PROD_CODE_30',
       'PROD_CODE_40', 'CUST_CODE', 'CUST_PRICE_SENSITIVITY', 'CUST_LIFESTAGE',
       'BASKET_ID', 'BASKET_SIZE', 'BASKET_PRICE_SENSITIVITY', 'BASKET_TYPE',
       'BASKET_DOMINANT_MISSION', 'STORE_CODE', 'STORE_FORMAT',
       'STORE_REGION'],
      dtype='object')

In [ ]:
# Feature Engineering RFM Analysis
# 1) Recency score [Last date = '20080706']

In [ ]:
df['R-score']=